In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

In [3]:
#Loading and reading the data
data = pd.read_csv('spam.csv',encoding='latin-1')

In [11]:

data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')


In [6]:
max_fatures = 2000

#Splitting the data and using Tokenizer
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)

X = pad_sequences(X)
print(X)

[[   0    0    0 ...   67   58  137]
 [   0    0    0 ...  443    6 1823]
 [   0    0    0 ...  459   79  382]
 ...
 [   0    0    0 ...   12   19  231]
 [   0    0    0 ...  198   12   50]
 [   0    0    0 ...    1   41  258]]


In [7]:
embed_dim = 128
lstm_out = 196

#Creating the model
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [8]:
model = createmodel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 152, 128)          256000    
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3733, 152) (3733, 2)
(1839, 152) (1839, 2)


In [10]:
batch_size = 32
model = createmodel()

#Obtaining X Train, Y Train, Xtest and Y Test
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2,validation_data=(X_test, Y_test))
model.save('spam.h5')

Epoch 1/5
117/117 - 97s - loss: 0.1573 - accuracy: 0.9478 - val_loss: 0.0733 - val_accuracy: 0.9799
Epoch 2/5
117/117 - 93s - loss: 0.0359 - accuracy: 0.9871 - val_loss: 0.0666 - val_accuracy: 0.9842
Epoch 3/5
117/117 - 93s - loss: 0.0166 - accuracy: 0.9957 - val_loss: 0.0718 - val_accuracy: 0.9842
Epoch 4/5
117/117 - 93s - loss: 0.0188 - accuracy: 0.9944 - val_loss: 0.0745 - val_accuracy: 0.9815
Epoch 5/5
117/117 - 93s - loss: 0.0101 - accuracy: 0.9968 - val_loss: 0.0759 - val_accuracy: 0.9826


In [12]:
#Printing the score and the accuracy
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

58/58 - 5s - loss: 0.0759 - accuracy: 0.9826
0.07588618248701096
0.9825992584228516
['loss', 'accuracy']
